In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-uwj0rzas
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-uwj0rzas
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=14c47b4a890e44110758a9d32a38c59d0a3a518a626f1ae58d124499e7a0a726
  Stored in directory: /tmp/pip-ephem-wheel-cache-impejs1h/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [12]:
!pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.4 MB/s eta 0:00:00


In [2]:
import torch
import clip
from transformers import AutoProcessor, AutoModel
import torch
from PIL import Image
from PIL import Image

In [3]:
# Load model
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [32]:
model, preprocess = clip.load("ViT-L/14", device=device)

In [34]:
image = preprocess(Image.open("/content/000000377368.jpg")).unsqueeze(0).to(device)

# Define choices
# choices = ["What is in the motorcyclist's mouth? - cigarette",
#            "What is in the motorcyclist's mouth? - food",
#            "What is in the motorcyclist's mouth? - popsicle stick",
#            "What is in the motorcyclist's mouth? - toothpick"]

# choices = ["a man with a toothpick in his mouth",
#            "a man eating food",
#            "a man with a popsicle stick in his mouth",
#            "a man with a cigarette in his mouth"]

# choices = ["motorcyclist mouth cigarette",
#            "motorcyclist mouth food",
#            "motorcyclist mouth popsicle stick",
#            "motorcyclist mouth toothpick"]
# choices = ["the motorcyclist's mouth has toothpick in it",
#  "the motorcyclist's mouth has food in it",
#  "the motorcyclist's mouth has popsicle stick in it",
#  "the motorcyclist's mouth has cigarette in it"]
# choices = ['top of the cupcakes has butter on it',
#  'top of the cupcakes has mayo on it',
#  'top of the cupcakes has ice cream on it',
#  'top of the cupcakes has icing on it']
# choices = ['the pool of water is frozen',
#  'the pool of water is fresh',
#  'the pool of water is dirty',
#  'the pool of water is boiling']
choices = ['The number is one',
 'The number is ten',
 'The number is nine',
 'The number is thirty']
# Encode image and text
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(clip.tokenize(choices).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (image_features @ text_features.T).squeeze(0)

# Get best choice
best_idx = similarity.argmax().item()
print("Predicted answer:", choices[best_idx])

Predicted answer: The number is one


In [35]:
similarity

tensor([0.1443, 0.1385, 0.1389, 0.1419])

In [36]:
import torch
import open_clip
from PIL import Image
from torchvision import transforms

# Load ViT-L/14 CLIP
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai')
tokenizer = open_clip.get_tokenizer('ViT-L-14')

/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


In [28]:
image = preprocess(Image.open("000000377368.jpg")).unsqueeze(0)

with torch.no_grad():
    img_features = model.encode_image(image)
    img_features /= img_features.norm(dim=-1, keepdim=True)

In [29]:
# labels = ['toothpick', 'food', 'popsicle stick', 'cigarette']
# labels = ['a man with a toothpick in his mouth',
#            'a man eating food',
#            'a man with a popsicle stick in his mouth',
#            'a man with a cigarette in his mouth']
# labels = ["motorcyclist's mouth? - cigarette",
#            "motorcyclist's mouth? - food",
#            "motorcyclist's mouth? - popsicle stick",
#            "motorcyclist's mouth? - toothpick"]
# labels = ["the motorcyclist's mouth has toothpick in it",
#  "the motorcyclist's mouth has food in it",
#  "the motorcyclist's mouth has popsicle stick in it",
#  "the motorcyclist's mouth has cigarette in it"]
# labels = ['top of the cupcakes has butter on it',
#  'top of the cupcakes has mayo on it',
#  'top of the cupcakes has ice cream on it',
#  'top of the cupcakes has icing on it']
# labels = ['the pool of water is frozen',
#  'the pool of water is fresh',
#  'the pool of water is dirty',
#  'the pool of water is boiling']
labels = ['The number is one',
 'The number is ten',
 'The number is nine',
 'The number is thirty']
text_tokens = tokenizer(labels)
with torch.no_grad():
    text_features = model.encode_text(text_tokens)
    text_features /= text_features.norm(dim=-1, keepdim=True)

# Compute similarity
sims = (img_features @ text_features.T)
best_label = labels[sims.argmax()]
print(sims, best_label)

tensor([[0.1387, 0.1373, 0.1317, 0.1442]]) The number is thirty


In [ ]:
model_name = "google/siglip-base-patch16-224"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

In [ ]:
image = Image.open("/content/000000461751.jpg")
texts = ["a man with a cigarette", "a man with food", "a man with a toothpick", "a man with a popsicle stick"]
inputs = processor(text=texts, images=[image]*len(texts), padding=True, return_tensors="pt")
outputs = model(**inputs)
scores = outputs.logits_per_image.softmax(dim=1)
print(texts[scores.argmax()])

a man with a toothpick


In [ ]:
scores

tensor([[1.3478e-02, 3.6653e-04, 5.5605e-01, 4.3011e-01],
        [1.3478e-02, 3.6653e-04, 5.5605e-01, 4.3011e-01],
        [1.3478e-02, 3.6653e-04, 5.5605e-01, 4.3011e-01],
        [1.3478e-02, 3.6653e-04, 5.5605e-01, 4.3011e-01]],
       grad_fn=<SoftmaxBackward0>)

In [ ]:
scores.argmax()

tensor(2)

In [9]:
# pip install transformers timm torch  (not shown)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import timm
import torch.nn as nn
import torch.nn.functional as F

# 1) Vision encoder (ViT-L/14) pretrained
vit = timm.create_model('vit_large_patch14_224', pretrained=True)
vit.patch_embed.eval()  # we will use forward to get patch tokens
# remove classifier head:
vit.reset_classifier(0)

# 2) Load SmolVLM (LLM part) and its tokenizer
llm_name = "HuggingFaceTB/SmolVLM-Instruct"  # replace with exact model id
tokenizer = AutoTokenizer.from_pretrained(llm_name, use_fast=False)
llm = AutoModelForCausalLM.from_pretrained(llm_name)
llm.eval()

# 3) Modality integrator: map ViT region feature -> LLM embedding size
LLM_EMB = llm.get_input_embeddings().weight.shape[1]
class ModalityAdapter(nn.Module):
    def __init__(self, in_dim, out_dim, n_vis_tokens=1):
        super().__init__()
        self.n_vis_tokens = n_vis_tokens
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.GELU(),
            nn.Linear(in_dim, out_dim * n_vis_tokens)
        )
        self.ln = nn.LayerNorm(out_dim)
    def forward(self, region_feat):  # region_feat: [B, C]
        x = self.mlp(region_feat)              # [B, out_dim * n_vis_tokens]
        x = x.view(region_feat.size(0), self.n_vis_tokens, -1)
        x = self.ln(x)
        return x  # [B, n_vis_tokens, out_dim]

# instantiate adapter
adapter = ModalityAdapter(in_dim=vit.embed_dim, out_dim=LLM_EMB, n_vis_tokens=2)

# 4) helper: pool patch tokens within a bbox
def image_to_patch_feats(image_tensor):  # image_tensor: [B,3,H,W], normalized for vit
    # use vit to get patch tokens (skip cls token); exact call depends on timm ViT API
    x = vit.patch_embed(image_tensor)    # [B, num_patches, embed_dim]
    # optionally forward transformer blocks if you want context-aware patches:
    # x = vit.pos_drop(vit.pos_embed + x)
    # x = vit.blocks(x) ...
    return x  # [B, P, C]

def bbox_to_region_feature(patch_feats, bbox, image_size=(224,224), patch_size=14):
    # bbox: [x1,y1,x2,y2] in pixel coords
    B, P, C = patch_feats.shape
    H, W = image_size
    ph, pw = patch_size, patch_size  # approximate
    # compute patch grid dimensions
    grid_h = H // ph
    grid_w = W // pw
    # convert bbox to patch grid indices (clamp)
    x1,y1,x2,y2 = bbox
    px1 = int((x1 / W) * grid_w)
    py1 = int((y1 / H) * grid_h)
    px2 = int((x2 / W) * grid_w)
    py2 = int((y2 / H) * grid_h)
    # unfold to [B, grid_h, grid_w, C]
    pgrid = patch_feats.view(B, grid_h, grid_w, C)
    # pool inside bbox
    region = pgrid[:, py1:py2+1, px1:px2+1, :].contiguous()
    pooled = region.view(B, -1, C).mean(dim=1)  # [B, C]
    return pooled

# 5) Build prompt and run LLM
def run_grounded_query(image_tensor, bbox, question_text):
    pf = image_to_patch_feats(image_tensor)  # [B, P, C]
    region_feat = bbox_to_region_feature(pf, bbox)  # [B, C]
    vis_tokens = adapter(region_feat)  # [B, n_vis, LLM_EMB]
    # create pseudo-token ids: we'll not feed ids but directly replace embeddings
    # tokenise question
    tokens = tokenizer(question_text, return_tensors='pt')
    input_ids = tokens["input_ids"]  # [B, T]
    # get text embeddings
    text_emb = llm.get_input_embeddings()(input_ids)  # [B, T, LLM_EMB]
    # prepend visual tokens (concatenate along sequence dim)
    full_emb = torch.cat([vis_tokens, text_emb], dim=1)  # [B, n_vis + T, LLM_EMB]

    # run the LLM with embeddings (many HF models accept inputs_embeds)
    outputs = llm(inputs_embeds=full_emb)
    # get decoder logits & sample text as usual (left for brevity)
    return outputs

# NOTE: for generation you'll need to implement a "virtual token" scheme where
# generation attention, position ids, and causal masking are set correctly.


RuntimeError: No pretrained weights exist for vit_large_patch14_224. Use `pretrained=False` for random init.